In [14]:
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.window as w
import pyspark.sql.types 

In [2]:
spark = SparkSession.builder.appName('mycourse').getOrCreate()

In [15]:
#define a custom schema
custome_schema = StructType([
    StructField('date_time', StringType(), True),
    StructField('userid', StringType(), True),
    StructField('domain', StringType(), True),
    StructField('dlbytes', IntegerType(), True),
    StructField('ulbytes', IntegerType(), True),
    StructField('clientip', StringType(), True),
    StructField('serverip', StringType(), True),
    StructField('country', StringType(), True),
    StructField('txn_time', FloatType(), True),
    StructField('http_method', StringType(), True),
    StructField('user_agent', StringType(), True),
    StructField('platform', StringType(), True)
])


# load the data with the custom schema
data_path = r'C:\Users\alex\Desktop\PySpark Crash Course Learn Spark Quickly\1 - Introduction\2 - course-file\course_file.csv'
df = spark.read.csv(data_path, schema=custome_schema, header=True)

# show the loaded Datafarme
df.show(5)

+-------------------+-------------+--------------------+-------+-------+---------------+-------------+--------+--------+-----------+--------------------+--------+
|          date_time|       userid|              domain|dlbytes|ulbytes|       clientip|     serverip| country|txn_time|http_method|          user_agent|platform|
+-------------------+-------------+--------------------+-------+-------+---------------+-------------+--------+--------+-----------+--------------------+--------+
|2023-10-04 11:37:11|7773153683656|    ryan-wells.co.uk| 872807| 741526|142.132.219.110|62.42.184.180| England|    2.33|      HTTPS|Mozilla/5.0 (Wind...| Android|
|2023-10-04 12:17:07|1886351675683|         hopkins.org|  50898| 529504|  184.205.48.78|152.123.41.39|   Wales|     1.2|       HTTP|Mozilla/5.0 (Wind...| Android|
|2023-10-02 23:25:12|1597721345356|           evans.com| 964276| 952420|  189.30.60.163|68.171.236.18|Scotland|    1.32|       HTTP|Mozilla/5.0 (Wind...|   Linux|
|2023-10-03 09:43:11|9

# window functions: rank, row_number, lead, lag, sum, avg

In [16]:
# rank
window_spec = w.Window.partitionBy('country').orderBy(f.col('dlbytes').desc())
df.withColumn('rank', f.rank().over(window_spec)).show(5)

+-------------------+-------------+-----------------+-------+-------+--------------+---------------+-------+--------+-----------+--------------------+--------+----+
|          date_time|       userid|           domain|dlbytes|ulbytes|      clientip|       serverip|country|txn_time|http_method|          user_agent|platform|rank|
+-------------------+-------------+-----------------+-------+-------+--------------+---------------+-------+--------+-----------+--------------------+--------+----+
|2023-10-04 01:21:13|3144906514401|         page.com| 999946| 769910|133.226.106.69|  43.150.184.67|  Wales|    2.73|       HTTP|Mozilla/5.0 (comp...| Windows|   1|
|2023-10-05 14:52:07|2736616479279|      watkins.com| 999912| 181045| 192.85.230.16|134.179.193.195|  Wales|    2.31|       HTTP|Mozilla/5.0 (comp...|     Mac|   2|
|2023-10-01 03:19:03|2236747991119|edwards-patel.com| 999910| 283407|  78.178.15.26|   132.242.86.8|  Wales|    1.92|       HTTP|Mozilla/5.0 (iPod...|     Mac|   3|
|2023-10-0

In [17]:
# row_number
window_spec = w.Window.partitionBy('country').orderBy('date_time')
df.withColumn('row_number', f.row_number().over(window_spec)).show(5)

+-------------------+-------------+----------------+-------+-------+---------------+--------------+-------+--------+-----------+--------------------+--------+----------+
|          date_time|       userid|          domain|dlbytes|ulbytes|       clientip|      serverip|country|txn_time|http_method|          user_agent|platform|row_number|
+-------------------+-------------+----------------+-------+-------+---------------+--------------+-------+--------+-----------+--------------------+--------+----------+
|2023-10-01 00:00:01|9935183354270|white-murray.biz| 196289| 359714|223.212.132.129|204.235.216.11|  Wales|    1.72|       HTTP|Mozilla/5.0 (comp...|     Mac|         1|
|2023-10-01 00:00:01|0434470039855| pritchard.co.uk| 350605| 860883|  70.236.57.196|33.243.179.115|  Wales|     0.4|       HTTP|Mozilla/5.0 (Maci...| Android|         2|
|2023-10-01 00:00:37|0253415742384|   wheeler.co.uk| 613899|  56659| 137.71.136.215|128.248.57.164|  Wales|    0.67|      HTTPS|Mozilla/5.0 (comp...| 

In [18]:
# lag
window_spec = w.Window.partitionBy().orderBy('date_time')
df.withColumn('previous_bytes', f.lag('dlbytes').over(window_spec)).show(5)

+-------------------+-------------+----------------+-------+-------+---------------+--------------+--------+--------+-----------+--------------------+--------+--------------+
|          date_time|       userid|          domain|dlbytes|ulbytes|       clientip|      serverip| country|txn_time|http_method|          user_agent|platform|previous_bytes|
+-------------------+-------------+----------------+-------+-------+---------------+--------------+--------+--------+-----------+--------------------+--------+--------------+
|2023-10-01 00:00:01|6242602789270|  ellis-hale.com|  60865| 378320|    65.219.1.13|     2.4.27.23| Ireland|    1.89|      HTTPS|Opera/9.61.(X11; ...|   Linux|          NULL|
|2023-10-01 00:00:01|9935183354270|white-murray.biz| 196289| 359714|223.212.132.129|204.235.216.11|   Wales|    1.72|       HTTP|Mozilla/5.0 (comp...|     Mac|         60865|
|2023-10-01 00:00:01|0434470039855| pritchard.co.uk| 350605| 860883|  70.236.57.196|33.243.179.115|   Wales|     0.4|       H

In [21]:
# lead
window_spec = w.Window.partitionBy().orderBy('date_time')
df.withColumn('next_bytes', f.lead('dlbytes').over(window_spec)).show(5)

+-------------------+-------------+----------------+-------+-------+---------------+--------------+--------+--------+-----------+--------------------+--------+----------+
|          date_time|       userid|          domain|dlbytes|ulbytes|       clientip|      serverip| country|txn_time|http_method|          user_agent|platform|next_bytes|
+-------------------+-------------+----------------+-------+-------+---------------+--------------+--------+--------+-----------+--------------------+--------+----------+
|2023-10-01 00:00:01|6242602789270|  ellis-hale.com|  60865| 378320|    65.219.1.13|     2.4.27.23| Ireland|    1.89|      HTTPS|Opera/9.61.(X11; ...|   Linux|    196289|
|2023-10-01 00:00:01|9935183354270|white-murray.biz| 196289| 359714|223.212.132.129|204.235.216.11|   Wales|    1.72|       HTTP|Mozilla/5.0 (comp...|     Mac|    350605|
|2023-10-01 00:00:01|0434470039855| pritchard.co.uk| 350605| 860883|  70.236.57.196|33.243.179.115|   Wales|     0.4|       HTTP|Mozilla/5.0 (Mac

In [25]:
# sum over
window_spec = w.Window.partitionBy('userid')
df.withColumn('sum_function', f.sum('dlbytes').over(window_spec)).show(5)

+-------------------+-------------+-------------+-------+-------+--------------+---------------+--------+--------+-----------+--------------------+--------+------------+
|          date_time|       userid|       domain|dlbytes|ulbytes|      clientip|       serverip| country|txn_time|http_method|          user_agent|platform|sum_function|
+-------------------+-------------+-------------+-------+-------+--------------+---------------+--------+--------+-----------+--------------------+--------+------------+
|2023-10-01 07:30:22|0000361494373|   kirby.info| 247353| 584248|115.60.168.113| 106.60.246.177| Ireland|    1.73|       HTTP|Mozilla/5.0 (X11;...| Windows|      483973|
|2023-10-03 13:55:48|0000361494373|   kirby.info| 236620| 423021|115.60.168.113| 106.60.246.177| England|    2.23|       HTTP|Mozilla/5.0 (X11;...| Windows|      483973|
|2023-10-02 10:45:43|0003860685467|   barton.com| 701021| 795588| 99.233.235.32|102.123.129.225| England|    2.02|       HTTP|Mozilla/5.0 (comp...| Wi

In [26]:
# avg over
window_spec = w.Window.partitionBy('domain')
df.withColumn('avg_function', f.avg('dlbytes').over(window_spec)).show(5)

+-------------------+-------------+--------------------+-------+-------+--------------+--------------+-------+--------+-----------+--------------------+--------+------------+
|          date_time|       userid|              domain|dlbytes|ulbytes|      clientip|      serverip|country|txn_time|http_method|          user_agent|platform|avg_function|
+-------------------+-------------+--------------------+-------+-------+--------------+--------------+-------+--------+-----------+--------------------+--------+------------+
|2023-10-05 12:11:28|3111478378469|abbott-chamberlai...| 835522| 512952| 209.44.242.39|172.94.129.111|England|    1.85|      HTTPS|Mozilla/5.0 (Wind...| Windows|    450770.0|
|2023-10-03 04:39:29|3111478378469|abbott-chamberlai...|  66018| 384673| 209.44.242.39|172.94.129.111|Ireland|    2.91|       HTTP|Mozilla/5.0 (Wind...| Windows|    450770.0|
|2023-10-02 23:43:07|3488034274431| abbott-mitchell.com| 175616| 396258| 63.111.97.111|191.135.117.14|Ireland|    2.84|      